# 背景概述

$\quad$**对映选择性**(enantioselectivity)是有机化学重要的研究主题之一. 在有机反应中, 手性催化剂的对映选择性往往与反应过渡态中一些重要的**非共价相互作用**(non-covalent interactions, NCIs)有关. 以[Birman等](https://pubs.acs.org/doi/10.1021/ja0491477)报道的酰基转移反应为例:

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/17993/6691c6d5302140c582b7343f24173c5d/bSUHb1WG1pR3NNhIsLuhDA.png)

在Birman等假设的机理中, 手性催化剂的对映选择性由$\pi$-$\pi$堆积或$\pi$-阳离子堆积作用介导: **特定手性的过渡态因位阻作用破坏了芳环的平行取向, 不利于堆积作用的形成**.

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/17993/e1edd4976ba44a58b0952756aeb950fd/RDAnx-oUTRUK7MbgTA6CPw.png)

$\quad$本次上机作业中, 我们将以线性回归模型定量地探索所用手性催化剂的性质对对映选择性的影响, 试图探索:
- 哪些性质参数会影响对映选择性? 
- 如何优化性质参数可以提升对映选择性 (例如减少还是增加某参数)?

为此, 我们将考察如下图所示的模型体系:

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/17993/ad92a2cb3d0040ffbde82802641d5a39/uz3674Qv_rCWWOAd9DpFTw.png)

其中, 底物的芳香环(**A**, **a**, **b**)按照特定的几何约束条件与催化剂(**D**, **c**, **d**)排列成复合物, 而芳环与底物的其余分子骨架(包含醇羟基和烷基)在图中箭头所示位置相连.

- 参考文献: [J. Am. Chem. Soc. 2017, 139, 20, 6803–6806](https://pubs.acs.org/doi/10.1021/jacs.7b02311)

# 数据探索与清洗

## 数据的初步探索

$\quad$在数据集`nci_birman.csv`中, 包含如下三组(用作特征的)参数:
- $\pi$-$\pi$堆积作用参数: `d_pi_d`, `d_pi_D`, `e_pi_d`, `e_pi_D`. 根据特定的电子结构计算方法(B97-D/def2TZVP水平), 对复合物进行几何优化, 给出几何结构信息与互作能(复合物能量减去各组分能量和). 其中:
  - 每组复合物中, 底物芳环由于具体朝向的不同, 存在2种可能构象(比如下图的例子), 分别以后缀`_d`和`_D`作区分;
    ![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/17993/842ed157585c4e4a8ba62ad2b91cf582/tUoBI8UPrb1b5S2Z1R3WHw.png)
  - 在计算中, 对底物与催化剂分子的两个芳环施加约束条件, 使二者完全“正对彼此”(环平面平行, 且两环中心的连线与环平面垂直). 此时, 我们找到使电子能量(前缀`e_`, 单位kcal/mol)最低的、也就是使复合物最稳定的环间距(前缀`d_`, 单位Å)
- 烷基几何参数: `L_Alk`, `B1_Alk`, `B5_Alk`, 分别代表长度、最小宽度、最大宽度.
- 芳环几何参数: `L_Ar`, `B1_Ar`, `B5_Ar`, 分别代表长度、最小宽度、最大宽度.
后两组参数称为[**Sterimol参数**](https://en.wikipedia.org/wiki/Sterimol_parameter), 其具体含义可从下图中做简单的理解:

![alt image.png](https://bohrium.oss-cn-zhangjiakou.aliyuncs.com/article/17993/842ed157585c4e4a8ba62ad2b91cf582/zpobYlZXNb5KHS_mwjLSBA.png)

$\quad$我们的线性回归模型将以上述参数作为特征(在化学信息学中, 这些特征也常称为**描述符**, descriptor), 试图预测特定催化剂针对特定底物的对映选择性. 在数据集中, 选择性以**对映比**$er$描述, 这是一个实验可测的量. 通过热力学公式, 我们可以将其与两种对映体的活化自由能之差$\Delta\Delta G^\ddagger$之间建立定量联系:

$$
\Delta\Delta G^\ddagger = -RT\ln{(er)}.
$$

## 数据的清洗与加工

$\quad$对于堆积相互作用参数, 我们不妨对其进行降维: 取两个可能构象`_d`和`_D`的参数加权和:
$$\begin{aligned}
D\pi_\mathrm{w} &:= c_\mathrm{d}D\pi_\mathrm{d} + c_\mathrm{D}D\pi_\mathrm{D},\\
E\pi_\mathrm{w} &:= c_\mathrm{d}E\pi_\mathrm{d} + c_\mathrm{D}E\pi_\mathrm{D},
\end{aligned}$$
其中, 权重$c_\mathrm{X}(\mathrm{X} = \mathrm{d}, \mathrm{D})$取为(按电子能量计算的)Boltzmann因子:
$$
c_\mathrm{X} = \frac{
\exp{\left(-\frac{E\pi_\mathrm{X}}{RT}\right)}
}{
\exp{\left(-\frac{E\pi_\mathrm{d}}{RT}\right)} + \exp{\left(-\frac{E\pi_\mathrm{D}}{RT}\right)}.
}
$$

**任务1**: 在`src/data.py`文件中, 编写函数`prepare_data()`, 完成数据的初步清洗.
  - 输入:
    - 直接读取自`csv`的原始数据集`in_df`;
    - 用于进行加权和与$\Delta \Delta G^\ddagger$计算的温度`temperature`, 以$\mathrm{K}$为单位.
  - 返回: 处理好的特征和标签. 其中:
    - 特征为`pd.DataFrame`格式, 包括:
      - 降维后的(加权)参数$D\pi_\mathrm{w}, E\pi_\mathrm{w}$, 以及二者的交叉项(乘积)$D\pi_\mathrm{w}E\pi_\mathrm{w}$, 分别命名为`d_pi_w`, `e_pi_w`与`de_pi_w`;
      - 所有原有的Sterimol几何参数, 名称不变.
    - 标签为`pd.Series`格式, 为自由能差$\Delta \Delta G^\ddagger = -RT\ln{(er)}$, 单位取为kcal/mol, 命名为`delta_delta_G`. 这里给出单位换算因子: $1 \mathrm{kcal/mol} = 4.184 \mathrm{kJ/mol}$.

### 提示
- 注意$er(\%)$一栏的单位, 例如表格中若该栏的值为$1$, 则实际代表$er = 1\%$, 或说$\Delta \Delta G^\ddagger = - RT \ln{0.01}$.

$\quad$完成任务1后, 可以运行下面的代码块对该函数进行初步测试.

In [10]:
!pytest -v -s tests/test_prepare_data.py

============================= test session starts =============================
platform win32 -- Python 3.12.10, pytest-8.4.2, pluggy-1.6.0 -- C:\Users\track\AppData\Local\Programs\Python\Python312\python.exe
cachedir: .pytest_cache
rootdir: d:\Work\Chemistry\课程\机器学习及其在化学中的应用\Homework\hw1
collecting ... collected 3 items

tests/test_prepare_data.py::test_prepare_data_names PASSED
tests/test_prepare_data.py::test_prepare_data_labels PASSED
tests/test_prepare_data.py::test_prepare_data_weighted_features PASSED

============================== 3 passed in 1.32s ==============================


## 特征的归一化

$\quad$随后, 进行数据的归一化(重标度)处理, 以使各个特征维度量纲一致.

$\quad$**任务2**: 在**各个数据集上**分别对特征作均值-方差归一化:

$$
x_n' := \frac{x_n - \mathrm{Mean}_{n=1}^N(x)}{\mathrm{Std}_{n=1}^N(x)},\,n=1,\dots,N,
$$

返回(归一化后的)训练集与测试集特征. 注意: **测试集上归一化时, 正确的做法是使用训练集上算出的均值和方差, 而非在测试集上重新计算**.

- **任务2.1**: 在`src/data.py`文件中, 编写函数`normalize()`, 完成数据集的特征归一化.
  - 输入:
    - 训练特征`X_train`、测试特征`X_test`.
  - 返回: 归一化后的训练特征`X_train_normalized`、测试特征`X_test_normalized`. 其中:
    - 对所有特征`X`进行均值-方差归一化;
    - **为了和`StandardScaler`保持一致, 请确保返回值均为`np.array`格式**, 例如对某`DataFrame`类型的变量`df`使用格式转换`np.array(df)`.
- **任务2.2**: 选出存在 **信息泄漏(information leak)** 或 **过拟合(overfitting)** 风险的一项: (a) 训练阶段将测试信息对模型可见; (b) 测试阶段将训练集信息对模型可见.

### 提示
- 归一化既可以像上机实习1演示的那样手动计算, 也可以通过函数[`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler)实现.
  - **注意**: 若选择`pandas`库手动计算...
    - 请在计算`std`时设置参数`ddof=0`, 以和`StandardScaler`方法保持一致.
    - 与上机实习稍有不同的是, 参与运算的是**每一列**的均值或标准差, 因此需要对[`mean()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html#pandas.DataFrame.mean)或[`std()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.std.html#pandas.DataFrame.std)等函数添加参数`axis`.
- 警惕机器学习模型在数据集上“作弊”.

$\quad$完成任务2.1后, 可以运行下面的代码块对该函数进行初步测试.

In [11]:
!pytest -v -s tests/test_normalize.py

============================= test session starts =============================
platform win32 -- Python 3.12.10, pytest-8.4.2, pluggy-1.6.0 -- C:\Users\track\AppData\Local\Programs\Python\Python312\python.exe
cachedir: .pytest_cache
rootdir: d:\Work\Chemistry\课程\机器学习及其在化学中的应用\Homework\hw1
collecting ... collected 2 items

tests/test_normalize.py::test_normalize_train PASSED
tests/test_normalize.py::test_normalize_consistency PASSED

============================== 2 passed in 3.53s ==============================


# 模型训练与评估

$\quad$我们在9个特征与23个训练样本上训练一个线性回归模型, 并评估其在5个测试样本上的预测表现. 我们可用[`sklearn.model_selection.train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn-model-selection-train-test-split)对数据集进行手动拆分, 训练 : 测试 = 23 : 5.

$\quad$**任务3**: 在`src/training.py`文件中, 搭建模型并完成训练与评估.

- **任务3.1**: 编写函数`train_model()`, 实现模型训练.
  - 输入: 训练集`X_train, y_true_train`.
  - 返回: 一个训练好的线性回归模型[`sklearn.linear_model.LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn-linear-model-linearregression).
- **任务3.2**: 编写函数`evaluate_model()`, 实现预测图、$R^2$与RMSE值的报告. 作图用的函数`plot_prediction()`已经给出.
  - 输入:
    - 模型`model`与数据集`X`, `y_true`;
    - 评估模式`mode`, 可在`plot`和`metrics`中二选一.
  - 返回: 如果评估模式为`plot`, 则作预测图、不返回任何内容; 如果评估模式为`metrics`, 则返回RMSE与$R^2$值.
- **任务3.3**: 分别给出训练集与测试集上的: RMSE值、$R^2$值、预测图象.

### 提示
$\quad$每完成一个子任务后, 你都可以运行下述代码块, 执行脚本文件`hw1.py`以检查输出. 完成任务3.1与3.2后, 请再运行一次下述代码块, 并在`results`目录下检查你的输出结果.

In [14]:
!python hw1.py --dataset data/nci_birman.csv --output-dir results/ --temperature 298.0 --test-size 5

      d_pi_w     e_pi_w    de_pi_w  L_Alk  B1_Alk  B5_Alk  L_Ar  B1_Ar  B5_Ar
0   3.540000 -10.240000 -36.249600   4.36    2.92    3.35  6.38   1.77   3.15
1   3.540000 -10.240000 -36.249600   4.35    2.09    3.34  6.38   1.77   3.15
2   3.540000 -10.240000 -36.249600   4.38    1.73    3.33  6.38   1.77   3.15
3   3.540000 -10.240000 -36.249600   3.08    1.70    2.20  6.38   1.77   3.15
4   3.490000 -11.880000 -41.461200   3.08    1.70    2.20  6.38   1.88   4.52
5   3.510000 -11.780000 -41.347800   3.08    1.70    2.20  6.63   1.83   4.41
6   3.481559 -13.154060 -45.796640   3.08    1.70    2.20  7.23   1.82   4.64
7   3.500000 -10.969124 -38.391934   3.08    1.70    2.20  6.38   1.77   4.71
8   3.560000 -13.940000 -49.626400   3.08    1.70    2.20  7.42   1.93   4.55
9   3.455769 -15.620682 -53.981468   3.08    1.70    2.20  6.41   1.77   5.62
10  3.540000 -14.300000 -50.622000   3.08    1.70    2.20  8.54   2.10   4.04
11  3.633040 -13.062459 -47.456432   3.08    1.70    2.20  8.58 

Traceback (most recent call last):
  File "d:\Work\Chemistry\课程\机器学习及其在化学中的应用\Homework\hw1\hw1.py", line 55, in <module>
    main(dataset, output_dir, temperature, test_size)
  File "d:\Work\Chemistry\课程\机器学习及其在化学中的应用\Homework\hw1\hw1.py", line 38, in main
    results.to_csv(os.path.join(output_dir, 'metrics.csv'), index=False)
  File "c:\Users\track\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\util\_decorators.py", line 333, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\track\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\generic.py", line 3986, in to_csv
    return DataFrameRenderer(formatter).to_csv(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\track\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\io\formats\format.py", line 1014, in to_csv
    csv_formatter.save()
  File "c:\Users\track\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\io\fo

# 结果分析

$\quad$**挑战任务**: 运行下面的代码块, 根据线性回归模型参数的正负, 考察各个因素对对映选择性的影响, 在注释的答题区中写下你的分析.
- $\pi$-$\pi$堆积作用如何影响对映选择性? (从几何结构与能量两方面考虑)
- 烷基的几何参数如何影响对映选择性? 和芳基几何参数相比, 总体看, 谁的影响更大?

### 提示
- 9列特征分别为:
  - `d_pi_w`, `e_pi_w`, `de_pi_w`, 表征$\pi$-$\pi$堆积结构的距离或能量参数;
  - `L_Alk`, `B1_Alk`, `B5_Alk`, 表征烷基的几何参数;
  - `L_Ar`, `B1_Ar`, `B5_Ar`, 表征芳基的几何参数.
- $\Delta \Delta G^\ddagger$越大, 表明两种对映异构体的动力学活性相差越大, 于是对映选择性越好.

In [15]:
import pickle
with open('results/lr.pkl', 'rb') as f:
    lr = pickle.load(f)
lr.coef_

array([ 0.30209862, -2.91947758,  2.63228439,  1.79148002,  0.19527638,
       -1.81569224, -0.04337466, -0.041071  , -0.15565019])

# 作业提交自查
在正式提交作业前, 请仔细核查是否包含了以下内容:
- `src`目录下代码模块完整的程序文件`data.py`、`training.py`;
- `results`目录下的输出结果: 线性回归模型系数`lr.pkl`、训练/测试集评估指标`metrics.csv`、预测回归图`pred_train.png` (训练集) 与`pred_test.png` (测试集).
- 选择题、问答题的答题纸`hw1.json`.